In [1]:
import pandas as pd
import json
from tqdm.auto import tqdm

In [2]:
with open('arabic_tweets_240229.json', 'r') as file:
    data = json.load(file)

df = pd.json_normalize(data)

In [3]:
pd.set_option('display.max_colwidth',None)
len(df)

1688253

### Filtering out non-Arab Users

In [4]:
# Removing texts written with harakat

def percentage_harakat(text):
    arabic_letters = 0
    harakat_count = 0
    harakat = ['َ', 'ُ', 'ِ', 'ْ', 'ّ', 'ً', 'ٌ', 'ٍ']  # List of harakat
    for char in text:
        if '\u0600' <= char <= '\u06FF' or '\u0750' <= char <= '\u077F' or '\u08A0' <= char <= '\u08FF' or '\uFB50' <= char <= '\uFDCF' or '\uFDF0' <= char <= '\uFDFF' or '\uFE70' <= char <= '\uFEFF':
            arabic_letters += 1
            if char in harakat:
                harakat_count += 1
    return (harakat_count / arabic_letters * 100) if arabic_letters > 0 else 0

# Filtering and printing deleted texts
to_delete = []
for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing"):
    text = row['tweet_text']
    if percentage_harakat(text) > 25:
        to_delete.append(index)
        #print(f"Deleted text: {text}")

# Dropping the rows from the DataFrame
df = df.drop(to_delete)
len(df)

Processing:   0%|          | 0/1688253 [00:00<?, ?it/s]

1634311

# Filtering-Out Media Channels etc

In [5]:
# Check the accounts with more than 500 tweets
user_id_counts = df['user_id'].value_counts()
user_ids_more_than_250 = user_id_counts[user_id_counts > 250].index.tolist()

# Initialize an empty list to store the results
selected_tweets = []

# For each user_id, select two random tweets
for user_id in user_ids_more_than_250:
    # Filter the DataFrame for the current user_id
    user_tweets = df[df['user_id'] == user_id]
    
    # Select two random tweets if available, or all tweets if less than two
    random_tweets = user_tweets.sample(n=min(1, len(user_tweets)), random_state=1)
    
    # Append these tweets to the selected_tweets list
    selected_tweets.extend(random_tweets.itertuples(index=False))

# Convert the list of namedtuples to a DataFrame for easier viewing/manipulation
selected_tweets_df = pd.DataFrame(selected_tweets)

# This DataFrame now contains two random tweets for each user with more than 250 tweets, along with user_id and screen_name
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
selected_tweets_df[['user_id', 'screen_name', 'tweet_text']]

,user_id,screen_name,tweet_text
0,3331605652,ScreenMix,الكويت تفتح صندوقاً لجمع التبرعات ب100 مليون دولار لإعادة إعمار غزة 🇵🇸🇰🇼
1,96083073,kasimf,فن الجواب\nفي زيارة للرئيس التركي رجب طيب أردوغان للجزائر ، أراد أحد الصحفيين إحراج أردوغان فسأله هل تعتبر أن الأتراك كانوا محتلين للجزائر ؟\n\nفرد عليه أردوغان : لو كان الأتراك دولة احتلال لكان سؤالك لي باللغة التركية وليس بالفرنسية ..\n\n😂😂😂😂
2,5536782,AJArabic,حكم لن يخفف من وجع العائلة.. خمس سنوات على الجريمة التي ماطل فيها القضاء الإسرائيلي ليصدر أخيرا حكما بالسجن المؤبد على المستوطن المدان بحرق عائلة دوابشة | نجوان سمري | الجزيرة | الضفة الغربية https://t.co/aqC3QOyDOh
3,242245578,Dr_alqarnee,الليلة ليلة 27 من رمضان هي أرجى الليالي أن تكون #ليلة_القدر ، فاجعل لك فيها أعمال صالحة من قيام ليل، ودُعاء، وتلاوة بتدبّر، وصدقة، وعفو، ومُسامحة، وبرّ، وصلة،\nأسأل الله أن يعتق رقابنا جميعاً من النار ، إنه العزيز الغفار.\n\n https://t.co/lSDbmuu85D
4,385291247,heekma,"من الرزق، أن يضع الله لك قبولاً في قلب كل من يراك."""
5,474886468,SaudiNews50,وزارة الصحة:\n\nحالات الاستثناء من لقاحات فيروس كورونا تكون بموجب تقارير طبية ترسل عبر البريد الإلكتروني.
6,347131861,TRTArabi,"بعد انتشار مقاطع فيديو تظهر الأوضاع ""السيئة"" للحجر الصحي في #مصر، والجدل الذي أثاره تصريح إعلامي مصري بقوله: ""الطيارة اللي جابتكو ترجعكو""، ما القصة؟ https://t.co/90dCIgwAAC"
7,380660175,,ماذا لو أنَّ دُعَاءنا اليوم\nهو قدرُنا غداً\nفاللهُمَّ ❤️
8,444958570,Hamza_tekin2023,صباح اليوم| ضيف أبيض لطيف يزور إسطنبول معانقا مسجد آيا صوفيا الكبير https://t.co/WUrvBoVc3x
9,60576219,,يا رب.. نسألك بقدرتك التي حفظت بها يونس في بطن الحوت.. ورحمتك التي شفيت بها أيوب قبل أن يموت.. ألا تبقِ لنا همًا ولا حُزنًا ولا ضيقًا ولا سقمًا إلا فرجته..


In [6]:
filtered_accounts= ['OrientNews', 'aa_arabic', 'turkistantuzbah', 'AJArabic',
                   'AJABreaking', 'ajmubasher', 'RTarabic', 'FilGoal',
                   'alsha3rawy', 'RassdNewsN', 'youm7', 'SkyNewsArabia_B', 'AlArabiya',
                   'Toonzatt', 'ScreenMix', 'heekma', 'SaudiNews50', 'TRTArabi',
                   'gorgeous4ew', 'forcabarca_ar', 'AlarabyTV', 'tcbestepe_ar', 
                   'syr_television', 'realmadridarab', 'muslim2day', 'AJELNEWS24', 
                   'tragency1', 'beINSPORTS', 'TurkeyCelebs', 'AlArabiya_Brk',
                   'CelebsArabic', 'skynewsarabia', 'rterdogan_ar', 'RTErdogan', 'Alhilal_FC',
                   '1G__h', 'ShehabAgency', 'llil2e', 'YeniSafakArabic',
                   'Sultanate_Om', 'DerkiNazir', 'TurkPressMedia', 'aawsat_News',
                   'arabqoute', 'bashar__asad', 'BBCArabic', 'fifaworldcup_ar',
                   'Arabi21News', 'SkyNewsArabia_B', 'ArabicBest', 'France24_ar',
                   'Psycholg', 'SyriaCivilDefe', 'TrendEPL', 'alaraby_ar',
                   'AlNassrFC', 'AlMayadeenNews', 'guzel_olalim']

In [7]:
df_filtered = df[~df['screen_name'].isin(filtered_accounts)]
len(df_filtered)

1589802

In [1]:
#df_filtered.to_excel('filtered_arabic_tweets_media_organs_and_turkish_users_excluded.xlsx')

# Stance towards Erdogan, Kılıçdaroğlu, Özdağ, Turks and Turkey

In [8]:
import re
import matplotlib.pyplot as plt

pattern_erd = r"أ?ردوغان"
erd_tweets = df_filtered[df_filtered['tweet_text'].apply(lambda x: bool(re.search(pattern_erd, x)))]

pattern_kk = r"كل[يى]جدار أ?و[غغ]لو"
kk_tweets= df_filtered[df_filtered['tweet_text'].apply(lambda x: bool(re.search(pattern_kk, x)))]

pattern_ozdag = r"أ?وزداغ"
ozdag_tweets= df_filtered[df_filtered['tweet_text'].apply(lambda x: bool(re.search(pattern_ozdag, x)))]

pattern_turkey = r"تركي[اى]"
turkey_tweets = df_filtered[df_filtered['tweet_text'].apply(lambda x: bool(re.search(pattern_turkey, x)))]

pattern_turkish = r"تركي[ىي]?"
turkish_tweets = df_filtered[df_filtered['tweet_text'].apply(lambda x: bool(re.search(pattern_turkish, x)))]

pattern_racism= r"عنصرية|عنصري"
racism_tweets = df_filtered[df_filtered['tweet_text'].apply(lambda x: bool(re.search(pattern_racism, x)))]

print(len(erd_tweets), len(kk_tweets), len(ozdag_tweets), len(turkey_tweets), len(turkish_tweets), len(racism_tweets))

11426 317 326 26704 44202 2874
